In [2]:
from src.board import *
from src.read_game import *
from src.parse_move import *
from src.board_position_notation import *

In [3]:
bkpt.output_notebook()

Loading BokehJS ...

In [4]:
b = Board()
bp = BoardPic(b)

In [5]:
bp.plot()

In [6]:
game = Game('data/game2.pgn')

In [11]:
print(game.current_move + 1)
parse_move(board=b, move=game.next_move, status=game.check_status)
bp.update(b)

1


In [14]:
game.game

{'metadata': {'Date': '',
  'Event': '',
  'North': '',
  'Result': '',
  'Round': '',
  'Site': '',
  'South': ''},
 'moves': [('A7>*', 'a6<*'),
  ('A5<', 'a3<'),
  ('A8<+', 'a5>'),
  ('A6>', 'a5>'),
  ('A3<', 'a8<'),
  ('A3>', 'a5>'),
  ('A5>', 'a7<*'),
  ('A3>', 'a3>'),
  ('A3<', 'a2<+'),
  ('A3>', 'a4<'),
  ('A3<', 'a2>*'),
  ('A3<', 'a7>'),
  ('A4<', 'a4<'),
  ('A7>', 'a4>'),
  ('A6>', 'a4>'),
  ('A2<', 'a4>'),
  ('A3>', 'a4>'),
  ('A3>', 'a4<'),
  ('A2<', 'a5<'),
  ('A7<', 'a1<'),
  ('A2<', 'a6>'),
  ('A2<', 'a3>'),
  ('A2>*', 'a2>'),
  ('B4<*', 'a5<'),
  ('A3<', 'a6<'),
  ('A4<', 'a4<*'),
  ('A2>', 'b1<'),
  ('A4<', 'b8>'),
  ('A2<*', 'a5<'),
  ('A3<*', 'a6<'),
  ('B6>', 'a7<'),
  ('B8>', 'b1<'),
  ('A3>', 'b2<'),
  ('A8>*', 'a1>'),
  ('A2>', 'b6>'),
  ('A6<', 'b7>'),
  ('B5>', 'b5<'),
  ('B2<', 'a2>'),
  ('B8>', 'b1<'),
  ('A3>', 'b7>'),
  ('B1<', 'b2<'),
  ('A4>', 'a3>'),
  ('A7<', 'b3<'),
  ('A2<*', 'a2>'),
  ('B3<*', 'a1>'),
  ('A1<*', 'b1<'),
  ('B7>', 'a8>*'),
  ('A1>', 'b